In [1]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

In [2]:
# 학습 데이터(MNIST)
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# X(이미지)와 Y(숫자값)의 입력값
X = tf.placeholder(tf.float32, [None, 28*28]) # 무한대 x 784(28*28) 행렬
Y = tf.placeholder(tf.int64, [None]) # 무한대 x 1 행렬

In [6]:
# CNN Model
def model(input_X):
    x_reshape = tf.reshape(X, [-1, 28, 28, 1])

    # Conv 레이어1
    W_conv1 = tf.Variable(tf.truncated_normal([5, 5, 1, 20], stddev=0.1)) # Filter가 Weight 역활을 함
    b_conv1 = tf.Variable(tf.zeros([20]))
    h_conv1 = tf.nn.conv2d(x_reshape, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1
    fmap_conv1 = tf.nn.tanh(h_conv1) # -> Feature(Activation) Map 생성

    # Pooling(Max) 레이어1
    h_pool1 = tf.nn.max_pool(fmap_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # fully-connected 레이어 1
    W_fc1 = tf.Variable(tf.truncated_normal([20 * 14 * 14, 500], stddev=0.1))
    b_fc1 = tf.Variable(tf.zeros([500]))
    h_pool1_flat = tf.reshape(h_pool1, [-1, 20 * 14 * 14])
    h_fc1 = tf.nn.tanh(tf.matmul(h_pool1_flat, W_fc1) + b_fc1)

    # 출력(fully connected) 레이어 2 (10개 출력)
    class_num = 10
    W_fc2 = tf.Variable(tf.truncated_normal([500, class_num], stddev=0.1))
    b_fc2 = tf.Variable(tf.zeros([class_num]))
    pred = tf.matmul(h_fc1, W_fc2) + b_fc2

    return pred

pred = model(X)

In [7]:
# Cost Function(Cross Entropy)
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=pred)
cost = tf.reduce_mean(cross_entropy)

# Gradient descent Optimizer(학습)
learning_rate = 0.01
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [8]:
# 학습 시작
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(10000):
    X_images, X_labels = mnist.train.next_batch(batch_size=100)
    X_labels = np.where(X_labels)[1]
#     print(X_labels)
        
    sess.run(optimizer, feed_dict={X: X_images, Y: X_labels})

    # 로그
    training_cost = sess.run(cost, feed_dict={X: X_images, Y: X_labels})
    print(epoch, training_cost)

print("학습완료! (cost : " + str(training_cost) + ")")

0 2.7482214
1 2.6815526
2 2.4703748
3 2.3028688
4 2.1172585
5 2.2668676
6 2.1859848
7 2.0734653
8 2.1431696
9 1.9512591
10 1.9493331
11 1.8829017
12 1.9441866
13 1.8445004
14 1.7390568
15 1.781153
16 1.7438452
17 1.7103789
18 1.6060945
19 1.6296124
20 1.534394
21 1.5328479
22 1.460444
23 1.4836441
24 1.6133536
25 1.5163257
26 1.5166162
27 1.4625809
28 1.5185869
29 1.4012526
30 1.404173
31 1.3958158
32 1.3095387
33 1.3362128
34 1.2582878
35 1.2189008
36 1.2604787
37 1.1548855
38 1.2369611
39 1.1869924
40 1.1790154
41 1.1650438
42 1.1911217
43 1.0678449
44 1.1774838
45 1.1168504
46 1.240674
47 1.0487227
48 1.0764993
49 1.0406132
50 1.0312394
51 1.0030335
52 1.0110747
53 1.0327803
54 1.0915619
55 1.0863999
56 1.002678
57 0.9621001
58 1.0042685
59 0.92264444
60 1.0675666
61 0.9469767
62 1.0107344
63 0.8517959
64 0.872988
65 0.96997917
66 0.8562586
67 0.91014636
68 0.98256457
69 0.8351291
70 0.96329033
71 0.78303057
72 0.73305404
73 0.7975708
74 0.7600269
75 0.7498891
76 0.84303737
77 0.801

576 0.19521555
577 0.36143836
578 0.30805558
579 0.3080242
580 0.33522302
581 0.33165357
582 0.28495696
583 0.32159528
584 0.27267703
585 0.31723508
586 0.2929624
587 0.38459128
588 0.33350986
589 0.39186025
590 0.37525174
591 0.30631316
592 0.33954975
593 0.26095763
594 0.24189548
595 0.35079858
596 0.39595807
597 0.23240146
598 0.31995723
599 0.45845658
600 0.2901366
601 0.30134302
602 0.22647978
603 0.32867137
604 0.25138152
605 0.31141686
606 0.2631386
607 0.20625347
608 0.28503844
609 0.41105628
610 0.3141516
611 0.20745291
612 0.37235558
613 0.41422558
614 0.37640586
615 0.18962486
616 0.24969162
617 0.27592617
618 0.33880764
619 0.30234137
620 0.3178662
621 0.3519068
622 0.32997155
623 0.4051373
624 0.35536107
625 0.30334014
626 0.29424784
627 0.35259816
628 0.39404178
629 0.44131377
630 0.3256311
631 0.32253295
632 0.28252354
633 0.36110458
634 0.29637125
635 0.29921448
636 0.36908737
637 0.31395167
638 0.39108
639 0.2387537
640 0.2450129
641 0.30906993
642 0.28548667
643 0.349

1127 0.32203034
1128 0.24660903
1129 0.2741027
1130 0.24211052
1131 0.2568897
1132 0.3163437
1133 0.25603774
1134 0.27950698
1135 0.231708
1136 0.21914971
1137 0.23694943
1138 0.29583794
1139 0.28539613
1140 0.27035153
1141 0.2315077
1142 0.22331509
1143 0.3324441
1144 0.24072066
1145 0.194372
1146 0.14082655
1147 0.26693025
1148 0.27604368
1149 0.19713178
1150 0.2605058
1151 0.2737453
1152 0.22184223
1153 0.31371248
1154 0.2084658
1155 0.397021
1156 0.23395865
1157 0.25766277
1158 0.22664541
1159 0.35510036
1160 0.21747221
1161 0.21423744
1162 0.24710527
1163 0.24008657
1164 0.23798399
1165 0.20682459
1166 0.2161188
1167 0.2766407
1168 0.16656125
1169 0.32080182
1170 0.30162323
1171 0.19328392
1172 0.15745692
1173 0.20629333
1174 0.2196726
1175 0.24665459
1176 0.21314289
1177 0.28414968
1178 0.31747127
1179 0.16318029
1180 0.28151944
1181 0.32919937
1182 0.20023975
1183 0.1765451
1184 0.30176795
1185 0.26906455
1186 0.22877419
1187 0.22453842
1188 0.19592355
1189 0.31890857
1190 0.351

1650 0.2711816
1651 0.14255215
1652 0.35496113
1653 0.16152221
1654 0.19499607
1655 0.21566631
1656 0.29182857
1657 0.2463024
1658 0.26315114
1659 0.18458107
1660 0.27607998
1661 0.26439476
1662 0.19792427
1663 0.25911555
1664 0.243679
1665 0.1483346
1666 0.17243578
1667 0.22985844
1668 0.14534916
1669 0.22908969
1670 0.14217976
1671 0.24598034
1672 0.25309798
1673 0.20541142
1674 0.2698606
1675 0.34791833
1676 0.19151424
1677 0.261624
1678 0.28297347
1679 0.19859585
1680 0.317141
1681 0.18835764
1682 0.2281733
1683 0.18202934
1684 0.22209717
1685 0.29409304
1686 0.15050796
1687 0.29996657
1688 0.207895
1689 0.10980126
1690 0.25992593
1691 0.17737792
1692 0.14088269
1693 0.24620481
1694 0.1397587
1695 0.23925945
1696 0.22134663
1697 0.26318452
1698 0.22976223
1699 0.2791648
1700 0.29096988
1701 0.16002347
1702 0.21267407
1703 0.22804265
1704 0.22808237
1705 0.12266623
1706 0.2759527
1707 0.22887674
1708 0.25844014
1709 0.18131173
1710 0.30983832
1711 0.2837713
1712 0.18562657
1713 0.31

2172 0.28552747
2173 0.22676876
2174 0.14031875
2175 0.23287958
2176 0.21362215
2177 0.24228829
2178 0.12247504
2179 0.22445534
2180 0.24295765
2181 0.29231268
2182 0.19389595
2183 0.19712362
2184 0.093558356
2185 0.14317764
2186 0.23434211
2187 0.25233018
2188 0.14823279
2189 0.2199711
2190 0.35196295
2191 0.2251508
2192 0.2481177
2193 0.24103087
2194 0.18527202
2195 0.2673515
2196 0.19363111
2197 0.12235417
2198 0.24443969
2199 0.26066333
2200 0.16225252
2201 0.20350349
2202 0.2794061
2203 0.11588812
2204 0.13776359
2205 0.2004725
2206 0.2547429
2207 0.087165825
2208 0.21272057
2209 0.30653077
2210 0.21392938
2211 0.23537515
2212 0.24230571
2213 0.2392419
2214 0.1040312
2215 0.23073955
2216 0.16191036
2217 0.17116265
2218 0.2580425
2219 0.20010151
2220 0.22465973
2221 0.4326254
2222 0.4384148
2223 0.13836373
2224 0.11414833
2225 0.20002691
2226 0.1981543
2227 0.22026964
2228 0.21033783
2229 0.16445585
2230 0.2522469
2231 0.2479925
2232 0.31175208
2233 0.25628963
2234 0.12662531
2235 

...

7842 0.11090766
7843 0.10047285
7844 0.12391508
7845 0.07969761
7846 0.14999966
7847 0.12975879
7848 0.10642097
7849 0.10436089
7850 0.107418545
7851 0.10465662
7852 0.099143505
7853 0.14111532
7854 0.10460431
7855 0.09975531
7856 0.0707406
7857 0.10679671
7858 0.092904165
7859 0.07331711
7860 0.05521156
7861 0.060970925
7862 0.076180294
7863 0.0875457
7864 0.054245625
7865 0.07239138
7866 0.05649984
7867 0.05827444
7868 0.13256699
7869 0.10216083
7870 0.13995636
7871 0.12050117
7872 0.36556166
7873 0.06194111
7874 0.09733376
7875 0.11540611
7876 0.05460964
7877 0.1081312
7878 0.087204754
7879 0.11181354
7880 0.149509
7881 0.16885626
7882 0.12993535
7883 0.10726128
7884 0.11492641
7885 0.17611967
7886 0.10727306
7887 0.07927915
7888 0.047222268
7889 0.0842808
7890 0.17871286
7891 0.065820225
7892 0.048030134
7893 0.044270385
7894 0.12833157
7895 0.08537847
7896 0.0854785
7897 0.16542678
7898 0.07438652
7899 0.09127567
7900 0.10029199
7901 0.09334597
7902 0.08579863
7903 0.13009499
7904

8352 0.12786865
8353 0.042489022
8354 0.09122797
8355 0.11686489
8356 0.084419556
8357 0.112080805
8358 0.044363007
8359 0.24743277
8360 0.027595356
8361 0.024213681
8362 0.0915681
8363 0.14181785
8364 0.08729622
8365 0.114485376
8366 0.16757074
8367 0.102066
8368 0.13543709
8369 0.12745582
8370 0.082954414
8371 0.059207514
8372 0.10721516
8373 0.11208399
8374 0.13691057
8375 0.16380945
8376 0.12622114
8377 0.17631763
8378 0.0527225
8379 0.05848541
8380 0.10408576
8381 0.05393081
8382 0.07933956
8383 0.14420079
8384 0.19229637
8385 0.063481435
8386 0.04768607
8387 0.05072131
8388 0.14337653
8389 0.21065468
8390 0.05091098
8391 0.14308359
8392 0.09763235
8393 0.12717049
8394 0.0502029
8395 0.10748522
8396 0.046980906
8397 0.14594093
8398 0.099588744
8399 0.15641738
8400 0.16390754
8401 0.061149277
8402 0.068453856
8403 0.08849134
8404 0.10200405
8405 0.09591117
8406 0.12368103
8407 0.12160367
8408 0.13835791
8409 0.12583438
8410 0.09942585
8411 0.15561682
8412 0.1989911
8413 0.12320759


8860 0.09974322
8861 0.093685225
8862 0.08221351
8863 0.08789717
8864 0.12615672
8865 0.10160177
8866 0.09374629
8867 0.060432468
8868 0.08391523
8869 0.04423277
8870 0.1365108
8871 0.06607265
8872 0.10963371
8873 0.12088831
8874 0.13635525
8875 0.06656661
8876 0.14641914
8877 0.096022226
8878 0.10334809
8879 0.14880072
8880 0.094599895
8881 0.10931087
8882 0.17705293
8883 0.111043654
8884 0.12955809
8885 0.16162075
8886 0.116024874
8887 0.11933654
8888 0.0960391
8889 0.036802426
8890 0.024863457
8891 0.07299569
8892 0.14065613
8893 0.181558
8894 0.17921565
8895 0.15299815
8896 0.092965335
8897 0.06680088
8898 0.07589411
8899 0.11146427
8900 0.14353919
8901 0.116974585
8902 0.11006436
8903 0.050988927
8904 0.16361855
8905 0.05626266
8906 0.06657937
8907 0.10316414
8908 0.07987988
8909 0.052590236
8910 0.066412896
8911 0.059250843
8912 0.049510006
8913 0.08783684
8914 0.146304
8915 0.1536249
8916 0.07959675
8917 0.114282496
8918 0.0455094
8919 0.096487015
8920 0.06279968
8921 0.06861792

9369 0.052152082
9370 0.108090945
9371 0.073773764
9372 0.09371236
9373 0.12784548
9374 0.1010955
9375 0.066753
9376 0.101250105
9377 0.15254976
9378 0.11809599
9379 0.109613225
9380 0.07169886
9381 0.084477976
9382 0.09011882
9383 0.07969179
9384 0.0920479
9385 0.09415598
9386 0.18697956
9387 0.10121985
9388 0.046680298
9389 0.1340707
9390 0.05603945
9391 0.09591282
9392 0.054377493
9393 0.17107946
9394 0.047687322
9395 0.048309144
9396 0.13093163
9397 0.09308199
9398 0.09060315
9399 0.12583698
9400 0.05676207
9401 0.0787754
9402 0.039925322
9403 0.0576148
9404 0.06514693
9405 0.13264455
9406 0.031388
9407 0.11407661
9408 0.0703217
9409 0.052432265
9410 0.07447308
9411 0.120880164
9412 0.06747414
9413 0.1329745
9414 0.11428332
9415 0.12842731
9416 0.11980657
9417 0.15831114
9418 0.06386273
9419 0.1488441
9420 0.12671319
9421 0.10038389
9422 0.10743046
9423 0.09007116
9424 0.10528124
9425 0.15597911
9426 0.1199545
9427 0.04847274
9428 0.08035509
9429 0.12749496
9430 0.16689634
9431 0.0

9878 0.13606793
9879 0.064061716
9880 0.15233234
9881 0.030180478
9882 0.04176373
9883 0.13902315
9884 0.057477627
9885 0.04494258
9886 0.06118139
9887 0.088866785
9888 0.0794833
9889 0.08003187
9890 0.110303745
9891 0.03593974
9892 0.08093167
9893 0.055488225
9894 0.1368962
9895 0.10711663
9896 0.045050368
9897 0.080939844
9898 0.058178738
9899 0.07565767
9900 0.1459013
9901 0.09591378
9902 0.05751914
9903 0.05266572
9904 0.11348296
9905 0.13851072
9906 0.051431414
9907 0.10416103
9908 0.16497953
9909 0.044961847
9910 0.062176738
9911 0.11456174
9912 0.047434278
9913 0.077299364
9914 0.05707478
9915 0.076502174
9916 0.04838286
9917 0.15064313
9918 0.11778454
9919 0.08699928
9920 0.050608158
9921 0.060215894
9922 0.07750031
9923 0.052038036
9924 0.121361755
9925 0.04407667
9926 0.06219149
9927 0.07380443
9928 0.06927371
9929 0.08712891
9930 0.124810725
9931 0.08480167
9932 0.08929286
9933 0.08302599
9934 0.11270091
9935 0.055582795
9936 0.09527026
9937 0.12389567
9938 0.15400922
9939 0

In [9]:
test_images, test_labels = mnist.test.next_batch(16)
test_labels = np.where(test_labels)[1]

batch_size = 100
nCnt = 0
for i in range(1):
    x_test, test_label = mnist.test.next_batch(batch_size)
    arr_data = sess.run(pred, feed_dict={X: x_test})

    pred_val = tf.argmax(arr_data, 1)
    real_val = np.where(test_label)[1]
    
    for ib in range(batch_size):
        tmp_pred = pred_val.eval(session=sess)
        tmp_real = real_val
        
        
        if tmp_pred[ib] == tmp_real[ib]:
            print("예측값: " + str(tmp_pred[ib]) + " / 실제값: " + str(tmp_real[ib]) + " => True")
            nCnt += 1
        else:
            print("예측값: " + str(tmp_pred[ib]) + " / 실제값: " + str(tmp_real[ib]) + " => False")

print("Accuracy:", (nCnt/100.0)*100)

예측값: 5 / 실제값: 5 => True
예측값: 8 / 실제값: 8 => True
예측값: 2 / 실제값: 2 => True
예측값: 8 / 실제값: 8 => True
예측값: 6 / 실제값: 6 => True
예측값: 6 / 실제값: 6 => True
예측값: 5 / 실제값: 5 => True
예측값: 1 / 실제값: 1 => True
예측값: 3 / 실제값: 3 => True
예측값: 9 / 실제값: 7 => False
예측값: 5 / 실제값: 5 => True
예측값: 0 / 실제값: 0 => True
예측값: 7 / 실제값: 7 => True
예측값: 5 / 실제값: 5 => True
예측값: 5 / 실제값: 5 => True
예측값: 7 / 실제값: 7 => True
예측값: 4 / 실제값: 4 => True
예측값: 4 / 실제값: 4 => True
예측값: 2 / 실제값: 2 => True
예측값: 8 / 실제값: 8 => True
예측값: 6 / 실제값: 6 => True
예측값: 9 / 실제값: 9 => True
예측값: 1 / 실제값: 1 => True
예측값: 6 / 실제값: 6 => True
예측값: 8 / 실제값: 8 => True
예측값: 3 / 실제값: 3 => True
예측값: 1 / 실제값: 1 => True
예측값: 5 / 실제값: 5 => True
예측값: 4 / 실제값: 4 => True
예측값: 1 / 실제값: 1 => True
예측값: 3 / 실제값: 3 => True
예측값: 1 / 실제값: 1 => True
예측값: 3 / 실제값: 3 => True
예측값: 2 / 실제값: 2 => True
예측값: 8 / 실제값: 8 => True
예측값: 5 / 실제값: 2 => False
예측값: 8 / 실제값: 8 => True
예측값: 4 / 실제값: 4 => True
예측값: 0 / 실제값: 0 => True
예측값: 3 / 실제값: 3 => True
예측값: 2 / 실제값: 2 => True
예측값: 3 / 실제값: 